In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
import tensorflow as tf
import json, gensim, sklearn, pickle, sys, re, os
import IPython.display as ipd

In [10]:
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.phrases import Phrases
from gensim.parsing.preprocessing import strip_tags, strip_short, strip_multiple_whitespaces, stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim import utils
from scholarly import scholarly
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from IPython.display import Audio
sound_file = './Music/invalid_keypress.mp3'

In [11]:
datapd=pd.read_json("arxiv-metadata-oai-snapshot.json",  lines=True)

In [5]:
wave = np.sin(2*np.pi*200*np.arange(5000*2)/10000) 
play=Audio(wave, rate=50000, autoplay=True)

## Model 1 -- TF-IDF ## 
### We will first focus on just one subfield -- hep-th ###

In [13]:
datapd_hep_th = datapd.loc[datapd['categories'].str.contains('hep-th')]
hep_abstracts = datapd_hep_th[ list(datapd_hep_th.iloc[:,0:1]) + ['abstract'] + ['title']]

In [17]:
datapd[0:5]

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [19]:
hep_abstracts[0:5]

,id,abstract,title
14,0704.0015,The pure spinor formulation of the ten-dimen...,Fermionic superstring loop amplitudes in the p...
17,0704.0018,We give a prescription for how to compute th...,In quest of a generalized Callias index theorem
51,0704.0052,We study space-time symmetries in scalar qua...,Quantum Field Theory on Curved Backgrounds. II...
63,0704.0064,We capture the off-shell as well as the on-s...,Nilpotent symmetry invariance in the superfiel...
65,0704.0066,Possible (algebraic) commutation relations i...,Lagrangian quantum field theory in momentum pi...


In [14]:
hep_corpus=hep_abstracts['abstract'].values
processed_hep=preprocess_documents(hep_corpus)

In [22]:
processed_hep[0:5]

[['pure',
  'spinor',
  'formul',
  'dimension',
  'superstr',
  'lead',
  'manifestli',
  'supersymmetr',
  'loop',
  'amplitud',
  'express',
  'integr',
  'pure',
  'spinor',
  'superspac',
  'paper',
  'explor',
  'differ',
  'method',
  'evalu',
  'integr',
  'us',
  'calcul',
  'kinemat',
  'factor',
  'loop',
  'loop',
  'massless',
  'point',
  'amplitud',
  'involv',
  'ramond',
  'state'],
 ['prescript',
  'comput',
  'callia',
  'index',
  'regul',
  'exponenti',
  'function',
  'agreement',
  'old',
  'result',
  'odd',
  'dimens',
  'problem',
  'comput',
  'dimens',
  'moduli',
  'space',
  'self',
  'dual',
  'string',
  'formul',
  'index',
  'problem',
  'dimension',
  'loop',
  'space',
  'think',
  'regul',
  'letter',
  'appli',
  'index',
  'problem'],
 ['studi',
  'space',
  'time',
  'symmetri',
  'scalar',
  'quantum',
  'field',
  'theori',
  'includ',
  'interact',
  'theori',
  'static',
  'space',
  'time',
  'consid',
  'euclidean',
  'quantum',
  'field',


In [15]:
hep_dictionary = gensim.corpora.Dictionary(processed_hep)
hep_bow_corpus = [hep_dictionary.doc2bow(text) for text in processed_hep]

In [30]:
print("Total number of papers in high energy physics theory (hep-th) : ",len(hep_bow_corpus))
print("Size of our dictionary : ", len(hep_dictionary))

Total number of papers in high energy physics theory (hep-th) :  140500
Size of our dictionary :  36210


In [31]:
hep_tfidf = gensim.models.TfidfModel(hep_bow_corpus, smartirs='npu')

In [32]:
index = gensim.similarities.MatrixSimilarity(hep_tfidf[hep_bow_corpus])

In [142]:
def search(string,number):
    new_doc = gensim.parsing.preprocessing.preprocess_string(string)
    new_vec = hep_dictionary.doc2bow(new_doc)
    vec_bow_tfidf = hep_tfidf[new_vec]
    sims = index[vec_bow_tfidf]
    for (s,i) in zip(sorted(enumerate(sims), key=lambda item: -item[1])[:number],range(1,number+1)):
        print( f"{i}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n")

In [147]:
search("ads cft entanglment islands information paradox",15)

1) Quantum Extremal Islands Made Easy, Part I: Entanglement on the Brane by Hong Zhe Chen, Robert C. Myers, Dominik Neuenfeld, Ignacio A. Reyes,
  Joshua Sandor (0.5870394). Arxiv: 2006.04851 

2) Islands outside the horizon by Ahmed Almheiri, Raghu Mahajan, Juan Maldacena (0.58141845). Arxiv: 1910.11077 

3) Islands in Asymptotically Flat 2D Gravity by Thomas Hartman, Edgar Shaghoulian, Andrew Strominger (0.56930244). Arxiv: 2004.13857 

4) Notes on islands in asymptotically flat 2d dilaton black holes by Takanori Anegawa, Norihiro Iizuka (0.54549414). Arxiv: 2004.01601 

5) Islands in cosmology by Thomas Hartman, Yikun Jiang, Edgar Shaghoulian (0.5453892). Arxiv: 2008.01022 

6) Entanglement islands in higher dimensions by Ahmed Almheiri, Raghu Mahajan, and Jorge E. Santos (0.5382712). Arxiv: 1911.09666 

7) Multipartite Purification, Multiboundary Wormholes and Islands in
  AdS$_{3}$/CFT$_{2}$ by Aranya Bhattacharya (0.50951785). Arxiv: 2003.11870 

8) Is the island universe model c

In [225]:
def recommend(string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    test_bow_corpus3 = [hep_dictionary.doc2bow(text) for text in processed_query3]
    vec_test_bow3=hep_tfidf[test_bow_corpus3]
    parameters={}
    for i in range(len(processed_query3)):
        parameters["sims" + str(i)] = index[vec_test_bow3[i]]
    values=parameters.values()
    for (s,j) in zip(sorted(enumerate(np.sqrt(sum(values))), key=lambda item: -item[1])[:number+len(processed_query3)],range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print( f" \n {j-len(processed_query3)}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]}")

In [226]:
recommend("1005.4690|1107.2116",15)

 
 1) Charged Dilatonic Black Holes and their Transport Properties by Blaise Gouteraux, Bom Soo Kim and Rene Meyer (0.9607182). Arxiv: 1102.4440
 
 2) Holography of electrically and magnetically charged black branes by Zhenhua Zhou, Jian-Pin Wu and Yi Ling (0.82526326). Arxiv: 1807.07704
 
 3) Magnetic Field Induced Quantum Criticality via new Asymptotically AdS_5
  Solutions by Eric D'Hoker and Per Kraus (0.76158416). Arxiv: 1006.2573
 
 4) Holography of Charged Dilaton Black Holes in General Dimensions by Chiang-Mei Chen and Da-Wei Pang (0.73488706). Arxiv: 1003.5064
 
 5) Criticality from EMD holography at finite temperature and density by Alfonso Ballon-Bayona, Henrique Boschi-Filho, Eduardo Folco Capossoli
  and Diego M. Rodrigues (0.71375096). Arxiv: 2006.08810
 
 6) Magnetic Critical Solutions in Holography by N. Angelinos (0.7114215). Arxiv: 1411.3579
 
 7) Phase transitions between Reissner-Nordstrom and dilatonic black holes
  in 4D AdS spacetime by Mariano Cadoni, Giuseppe D

## Model 2 -- Doc2Vec ##
### Again, just focusing on hep-th ###

In [265]:
def testmodel(model,phrase,number):
    new_doc = gensim.parsing.preprocessing.preprocess_string(phrase)
    test_doc_vector = model.infer_vector(new_doc)
    sims = model.docvecs.most_similar(positive = [test_doc_vector],topn=number)
    for (s,i) in zip(sims,range(1,number+1)):
        print( f"{i}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n") 

In [285]:
def recommend_doc2vec(model,string,number):
    pointer3=hep_abstracts.loc[(hep_abstracts['id'].str.match(string))]
    queryvalue3=pointer3['abstract'].values
    processed_query3=preprocess_documents(queryvalue3)
    temper=[[] for y in range(len(processed_query3))]
    for i in range(len(processed_query3)):
        temper[i]=model.infer_vector(processed_query3[i])
    sims = model.docvecs.most_similar(positive = temper,topn=1+number+len(processed_query3))
    for (s,j) in zip(sims,range(1,number+1+len(processed_query3))):
        if j<=len(processed_query3):
            continue
        print( f"{j-len(processed_query3)}) {datapd_hep_th['title'].iloc[s[0]]} by {datapd_hep_th['authors'].iloc[s[0]]} ({str(s[1])}). Arxiv: {datapd_hep_th['id'].iloc[s[0]]} \n") 

In [287]:
tagged_hep_corpus = [TaggedDocument(d, [i]) for i, d in enumerate(processed_hep)]

In [302]:
model_big_bow300_5_10 = Doc2Vec(tagged_hep_corpus, dm=0, vector_size=300, window=5, min_count=1, epochs=10, hs=1)

In [303]:
recommend_doc2vec(model_big_bow300_5_10,"1212.4103|1212.3616",10)

1) Conformal Field Theories in Fractional Dimensions by S. El-Showk, M. Paulos, D. Poland, S. Rychkov, D. Simmons-Duffin, A.
  Vichi (0.45039984583854675). Arxiv: 1309.5089 

2) Analytic bootstrap at large spin by Apratim Kaviraj, Kallol Sen and Aninda Sinha (0.4402136206626892). Arxiv: 1502.01437 

3) Bootstrapping the O(N) Vector Models by Filip Kos, David Poland, David Simmons-Duffin (0.4221642017364502). Arxiv: 1307.6856 

4) Higher Spin ANEC and the Space of CFTs by David Meltzer (0.42161595821380615). Arxiv: 1811.01913 

5) Fast Conformal Bootstrap and Constraints on 3d Gravity by Nima Afkhami-Jeddi, Thomas Hartman, and Amirhossein Tajdini (0.4142104685306549). Arxiv: 1903.06272 

6) Correlation functions in scalar field theory at large charge by Guillermo Arias-Tamargo, Diego Rodriguez-Gomez, Jorge G. Russo (0.41208416223526). Arxiv: 1912.01623 

7) Conformal Field Theories and Deep Inelastic Scattering by Zohar Komargodski, Manuela Kulaxizi, Andrei Parnachev, and Alexander
  Zh

In [306]:
testmodel(model_big_bow300_5_10,"sparse conformal field theories holgraphy",10)

1) Conformal Field Theory Correlators From sine-Gordon Model on AdS
  Spacetime by Sze-Shiang Feng and Mu-Lin Yan (0.49992844462394714). Arxiv: hep-th/0008179 

2) An Introduction to Conformal Field Theory by Matthias R Gaberdiel (0.4996393322944641). Arxiv: hep-th/9910156 

3) Fermionisation of a Two-Dimensional Free Massless Complex Scalar Field by Laure Gouba (IMSP, CIPMA, Rep. Benin), Gabriel Y.H. Avossevou (IMSP,
  CIPMA, Rep. Benin), Jan Govaerts (UCL, Louvain-la-Neuve, Belgium), M. Norbert
  Hounkonnou (IMSP, CIPMA, Rep. Benin) (0.49109622836112976). Arxiv: hep-th/0408024 

4) Operator Product Expansions and Consistency Relations in a O(N)
  Invariant Fermionic CFT for 2<d<4 by Anastasios C. Petkou (0.4671935439109802). Arxiv: hep-th/9602054 

5) Logarithmic conformal field theories and AdS correspondence by A.M. Ghezelbash. M. Khorrami, A. Aghamohammadi (0.4661291241645813). Arxiv: hep-th/9807034 

6) On the BCFT Description of Holes in the c=1 Matrix Model by Davide Gaiotto, N

## Testing different models ##

### We found that CBOW works much better than skipgram for a given windowsize, vector dimensions and epochs. ###

In [293]:
model_big_bow200_2_10 = pickle.load(open("Modelsfeb3/model_big_bow200_2_10", 'rb'))
model_big_bow200_2_20 = pickle.load(open("Modelsfeb3/model_big_bow200_2_20", 'rb'))
model_big_bow200_2_30 = pickle.load(open("Modelsfeb3/model_big_bow200_2_30", 'rb'))
model_big_bow200_3_10 = pickle.load(open("Modelsfeb3/model_big_bow200_3_10", 'rb'))
model_big_bow200_3_20 = pickle.load(open("Modelsfeb3/model_big_bow200_3_20", 'rb'))
model_big_bow200_3_30 = pickle.load(open("Modelsfeb3/model_big_bow200_3_30", 'rb'))
model_big_bow200_4_10 = pickle.load(open("Modelsfeb3/model_big_bow200_4_10", 'rb'))
model_big_bow200_4_20 = pickle.load(open("Modelsfeb3/model_big_bow200_4_20", 'rb'))

In [294]:
model_big_bow300_2_10 = pickle.load(open("Modelsfeb3/model_big_bow300_3_10", 'rb'))
model_big_bow300_2_20 = pickle.load(open("Modelsfeb3/model_big_bow300_3_20", 'rb'))
model_big_bow300_2_30 = pickle.load(open("Modelsfeb3/model_big_bow300_3_30", 'rb'))
model_big_bow300_3_10 = pickle.load(open("Modelsfeb3/model_big_bow300_3_10", 'rb'))
model_big_bow300_3_20 = pickle.load(open("Modelsfeb3/model_big_bow300_3_20", 'rb'))
model_big_bow300_3_30 = pickle.load(open("Modelsfeb3/model_big_bow300_3_30", 'rb'))
model_big_bow300_4_10 = pickle.load(open("Modelsfeb3/model_big_bow300_4_10", 'rb'))
model_big_bow300_4_20 = pickle.load(open("Modelsfeb3/model_big_bow300_4_20", 'rb'))
model_big_bow300_5_20 = pickle.load(open("Modelsfeb3/model_big_bow300_5_20", 'rb'))
model_big_bow300_5_30 = pickle.load(open("Modelsfeb3/model_big_bow300_5_30", 'rb'))

In [298]:
model_big_skipgram_200_2_20 = pickle.load(open("Modelsfeb3/model_big_skipgram200_2_20", 'rb'))
model_big_skipgram_200_2_30 = pickle.load(open("Modelsfeb3/model_big_skipgram200_2_30", 'rb'))
model_big_skipgram_200_2_30 = pickle.load(open("Modelsfeb3/model_big_skipgram200_2_30", 'rb'))
model_big_skipgram_200_4_10 = pickle.load(open("Modelsfeb3/model_big_skipgram200_4_10", 'rb'))
model_big_skipgram_200_4_20 = pickle.load(open("Modelsfeb3/model_big_skipgram200_4_20", 'rb'))

In [301]:
model_big_skipgram_300_2_20 = pickle.load(open("Modelsfeb3/model_big_skipgram300_2_20", 'rb'))
model_big_skipgram_300_2_30 = pickle.load(open("Modelsfeb3/model_big_skipgram300_2_30", 'rb'))
model_big_skipgram_300_3_20 = pickle.load(open("Modelsfeb3/model_big_skipgram300_3_20", 'rb'))
model_big_skipgram_300_3_30 = pickle.load(open("Modelsfeb3/model_big_skipgram300_3_30", 'rb'))
model_big_skipgram_300_4_10 = pickle.load(open("Modelsfeb3/model_big_skipgram300_4_10", 'rb'))
model_big_skipgram_300_4_20 = pickle.load(open("Modelsfeb3/model_big_skipgram300_4_20", 'rb'))

# Our (not definitive, still testing) conclusion -- CBOW with vector dimensions = 300 and window=5, works best #